In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

import os
import glob

# Part I: Edgelist csv

## 1 Load Data 

In [2]:
# Load data
df = pd.read_excel('2019UNData.xlsx', sheet_name = 'Table 1', header = 14)
df.head(2)

Year  Sort\norder Major area, region, country or area of destination  \
0     NaN          NaN                                                NaN   
1  1990.0    1990001.0                                              WORLD   

  Notes   Code Type of data (a) Unnamed: 6   Unnamed: 7   Unnamed: 8  \
0   NaN    NaN              NaN      Total  Other South  Other North   
1   NaN  900.0              NaN  153011473      6548526      2366800   

  Country or area of origin     ...      Unnamed: 231 Unnamed: 232  \
0               Afghanistan     ...           Uruguay   Uzbekistan   
1                   6823350     ...            237486      1428020   

  Unnamed: 233                        Unnamed: 234 Unnamed: 235  \
0      Vanuatu  Venezuela (Bolivarian Republic of)     Viet Nam   
1         5060                              185946      1237873   

                Unnamed: 236    Unnamed: 237 Unnamed: 238 Unnamed: 239  \
0  Wallis and Futuna Islands  Western Sahara        Yemen       Zambia   
1                       6484          168239       455492        85203   

  Unnamed: 240  
0     Zimbabwe  
1       204365  

[2 rows x 241 columns]

In [3]:
list(df.columns)

['Year',
 'Sort\norder',
 'Major area, region, country or area of destination',
 'Notes',
 'Code',
 'Type of data (a)',
 'Unnamed: 6',
 'Unnamed: 7',
 'Unnamed: 8',
 'Country or area of origin',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 'Unnamed: 18',
 'Unnamed: 19',
 'Unnamed: 20',
 'Unnamed: 21',
 'Unnamed: 22',
 'Unnamed: 23',
 'Unnamed: 24',
 'Unnamed: 25',
 'Unnamed: 26',
 'Unnamed: 27',
 'Unnamed: 28',
 'Unnamed: 29',
 'Unnamed: 30',
 'Unnamed: 31',
 'Unnamed: 32',
 'Unnamed: 33',
 'Unnamed: 34',
 'Unnamed: 35',
 'Unnamed: 36',
 'Unnamed: 37',
 'Unnamed: 38',
 'Unnamed: 39',
 'Unnamed: 40',
 'Unnamed: 41',
 'Unnamed: 42',
 'Unnamed: 43',
 'Unnamed: 44',
 'Unnamed: 45',
 'Unnamed: 46',
 'Unnamed: 47',
 'Unnamed: 48',
 'Unnamed: 49',
 'Unnamed: 50',
 'Unnamed: 51',
 'Unnamed: 52',
 'Unnamed: 53',
 'Unnamed: 54',
 'Unnamed: 55',
 'Unnamed: 56',
 'Unnamed: 57',
 'Unnamed: 58',
 'Unnamed: 59',
 'Unn

## 2 Clean dataframe

In [4]:
# Clean df

# Delete unnecessary columns [1:8]
df = df.drop(columns=['Sort\norder', "Notes", "Code", "Type of data (a)", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8"], axis=1)

# Change column names using values from row 0
df.columns = df.iloc[0]

# Adjust the first two column names from NAN to "Year" and "Destination" and drop the redundant row 0
list1 = list(df.iloc[0])
list1 = list1[2:]
list2 = ["Year", "Destination"]
list3 = list2 + list1
df.columns = list3
df = df.drop([0])

# Convert column 'Year' to int type
df['Year'] = df['Year'].astype(int)

# impute NAN with 0
df = df.fillna(0)

# select only country data not regional or income group data
list_country = list(df.columns)
list_country = list_country[2:]
df = df.loc[df['Destination'].isin(list_country)]

df.head(3)

Year Destination Afghanistan Albania Algeria American Samoa Andorra  \
24  1990     Burundi           0       0       0              0       0   
25  1990     Comoros           0       0       0              0       0   
26  1990    Djibouti           0       0       0              0       0   

   Angola Anguilla Antigua and Barbuda   ...    Uruguay Uzbekistan Vanuatu  \
24      0        0                   0   ...          0          0       0   
25      0        0                   0   ...          0          0       0   
26      0        0                   0   ...          0          0       0   

   Venezuela (Bolivarian Republic of) Viet Nam Wallis and Futuna Islands  \
24                                  0        0                         0   
25                                  0        0                         0   
26                                  0        0                         0   

   Western Sahara Yemen Zambia Zimbabwe  
24              0     0      0        0  
25              0     0      0        0  
26              0   289      0        0  

[3 rows x 234 columns]

In [5]:
df.isnull().sum().sum()

0

## 3 Generate yearly data

In [6]:
df_k = df[df['Year'] == 2015]
df_k = df_k[df_k['Destination'] == 'Angola']
df_k["Côte d'Ivoire"]

1460    46
Name: Côte d'Ivoire, dtype: object

### 3.1 2019 data 

In [7]:
def generate_yearly_data(dataframe, year, country_list, weight_name, csv_file_name):
    
    # make a copy first
    df_year = dataframe.copy()
    
    # select specific year
    df_year = dataframe.loc[dataframe['Year'] == year]
    
    # select rows that are countries in the contry_list
    df_year = df_year.loc[df_year['Destination'].isin(country_list)]
    
    # reset index
    df_year = df_year.reset_index(drop = True)
    
    # adjust column name Year to Destination
    df_year_t = df_year.drop(columns = ['Year']).set_index('Destination')
    
    # reorder the index according to the country name orders defined in the hearder
    df_year_t = df_year_t.reindex(country_list)
    
    # transform matrix format to edgelist format
    df_year_edgelist = df_year_t.stack().reset_index()
    
    # rename hearder
    df_year_edgelist = df_year_edgelist.rename(columns = {"level_1": "Origin", 0:weight_name})
    
    # remove rows that is similar to the format of "Afghanistan to Afganistan"
    indexnames = df_year_edgelist[df_year_edgelist['Destination'] == df_year_edgelist['Origin']].index
    df_year_edgelist.drop(indexnames, inplace=True)
    df_year_edgelist = df_year_edgelist.reset_index(drop = True)
    
    # convert flow column from object type to int type for later convenience 
    try:
        df_year_edgelist[weight_name] = df_year_edgelist[weight_name].astype(float)
    except:
        df_year_edgelist[weight_name] = pd.to_numeric(df_year_edgelist[weight_name], errors='coerce')
        df_year_edgelist[weight_name] = df_year_edgelist[weight_name].astype(float)
    
    # export csv file
    #df_year_edgelist.to_csv(csv_file_name, index=False)
    
    # return df
    return df_year_edgelist

In [8]:
df_2019 = generate_yearly_data(df, int(2019), list_country, "flow2019", "MigrationData05_19/EdgeList2019.csv")

In [9]:
df_2019.head()

Destination          Origin  flow2019
0  Afghanistan         Albania       0.0
1  Afghanistan         Algeria       0.0
2  Afghanistan  American Samoa       0.0
3  Afghanistan         Andorra       0.0
4  Afghanistan          Angola       0.0

In [10]:
df_2019.dtypes

Destination     object
Origin          object
flow2019       float64
dtype: object

In [11]:
df_2019['flow2019'].isnull().sum()

0

### 3.2 2015 Data 

In [12]:
df_2015 = generate_yearly_data(df, int(2015), list_country, "flow2015", "MigrationData05_19/EdgeList2015.csv")
df_2015.head()

Destination          Origin  flow2015
0  Afghanistan         Albania       0.0
1  Afghanistan         Algeria       0.0
2  Afghanistan  American Samoa       0.0
3  Afghanistan         Andorra       0.0
4  Afghanistan          Angola       0.0

In [13]:
df_2015.dtypes

Destination     object
Origin          object
flow2015       float64
dtype: object

In [14]:
df_2015['flow2015'].isnull().sum()

0

### 3.3 2010 Data 

In [15]:
df_2010 = generate_yearly_data(df, int(2010), list_country, "flow2010", "MigrationData05_19/EdgeList2010.csv")
df_2010.head()

Destination          Origin  flow2010
0  Afghanistan         Albania       0.0
1  Afghanistan         Algeria       0.0
2  Afghanistan  American Samoa       0.0
3  Afghanistan         Andorra       0.0
4  Afghanistan          Angola       0.0

In [16]:
df_2010['flow2010'].isnull().sum()

3

In [17]:
df_2010[df_2010.isnull().any(axis=1)]

Destination       Origin  flow2010
26065       Kuwait  South Sudan       NaN
44580      Somalia        Yemen       NaN
45011  South Sudan        Sudan       NaN

In [18]:
df_2010 = df_2010.fillna(0)
df_2010['flow2010'].isnull().sum()

0

### 3.3 2005 Data 

In [19]:
df_2005 = generate_yearly_data(df, int(2005), list_country, "flow2005", "MigrationData05_19/EdgeList2005.csv")
df_2005.head()

Destination          Origin  flow2005
0  Afghanistan         Albania       0.0
1  Afghanistan         Algeria       0.0
2  Afghanistan  American Samoa       0.0
3  Afghanistan         Andorra       0.0
4  Afghanistan          Angola       0.0

In [20]:
df_2005['flow2005'].isnull().sum()

16

In [21]:
df_2005[df_2005.isnull().any(axis=1)]

Destination                            Origin  flow2005
26065       Kuwait                       South Sudan       NaN
31879   Montenegro                           Albania       NaN
44580      Somalia                             Yemen       NaN
44853  South Sudan          Central African Republic       NaN
44854  South Sudan                              Chad       NaN
44872  South Sudan  Democratic Republic of the Congo       NaN
44878  South Sudan                             Egypt       NaN
44881  South Sudan                           Eritrea       NaN
44884  South Sudan                          Ethiopia       NaN
44912  South Sudan                             India       NaN
44924  South Sudan                             Kenya       NaN
44933  South Sudan                             Libya       NaN
44941  South Sudan                              Mali       NaN
44965  South Sudan                           Nigeria       NaN
45011  South Sudan                             Sudan       NaN
45028  South Sudan                            Uganda       NaN

In [22]:
df_2005 = df_2005.fillna(0)
df_2005['flow2005'].isnull().sum()

0

### 3.4 2000 Data 

In [23]:
df_2000 = generate_yearly_data(df, int(2000), list_country, "flow2000", "MigrationData05_19/EdgeList2000.csv")
df_2000.head()

Destination          Origin  flow2000
0  Afghanistan         Albania       0.0
1  Afghanistan         Algeria       0.0
2  Afghanistan  American Samoa       0.0
3  Afghanistan         Andorra       0.0
4  Afghanistan          Angola       0.0

In [24]:
df_2000['flow2000'].isnull().sum()
df_2000[df_2000.isnull().any(axis=1)].head()

Destination   Origin  flow2000
3946    Barbados   Belize       NaN
3948    Barbados  Bermuda       NaN
3969    Barbados    China       NaN
3979    Barbados     Cuba       NaN
4019    Barbados    Haiti       NaN

In [25]:
df_2000 = df_2000.fillna(0)
df_2000['flow2000'].isnull().sum()

0

## 4 Calculate increase rate 

In [26]:
# merge all flow data into one df
df_all = df_2000.copy()
df_all['flow2005'] = df_2005['flow2005']
df_all['flow2010'] = df_2010['flow2010']
df_all['flow2015'] = df_2015['flow2015']
df_all['flow2019'] = df_2019['flow2019']
df_all.head()

Destination          Origin  flow2000  flow2005  flow2010  flow2015  \
0  Afghanistan         Albania       0.0       0.0       0.0       0.0   
1  Afghanistan         Algeria       0.0       0.0       0.0       0.0   
2  Afghanistan  American Samoa       0.0       0.0       0.0       0.0   
3  Afghanistan         Andorra       0.0       0.0       0.0       0.0   
4  Afghanistan          Angola       0.0       0.0       0.0       0.0   

   flow2019  
0       0.0  
1       0.0  
2       0.0  
3       0.0  
4       0.0

In [27]:
df_all.dtypes

Destination     object
Origin          object
flow2000       float64
flow2005       float64
flow2010       float64
flow2015       float64
flow2019       float64
dtype: object

### 4.1 2000 - 2005 increase rate 

In [28]:
# Calcualte migration flow increase rate
df_all['increase_rate_2000_2005'] = 100 * ((df_all['flow2005'] - df_all['flow2000'])/df_all['flow2000'])
df_all['increase_rate_2000_2005'] = df_all['increase_rate_2000_2005'].fillna(0)

In [29]:
df_all.loc[~np.isfinite(df_all['increase_rate_2000_2005']), 'increase_rate_2000_2005'] = float(0)

### 4.1 2005 - 2010 increase rate

In [30]:
# Calcualte migration flow increase rate
df_all['increase_rate_2005_2010'] = 100 * ((df_all['flow2010'] - df_all['flow2005'])/df_all['flow2005'])
df_all['increase_rate_2005_2010'] = df_all['increase_rate_2005_2010'].fillna(0)

In [31]:
df_all.loc[~np.isfinite(df_all['increase_rate_2005_2010']), 'increase_rate_2005_2010'] = float(0)

### 4.2 2010 - 2015 increase rate

In [32]:
# Calcualte migration flow increase rate
df_all['increase_rate_2010_2015'] = 100 * ((df_all['flow2015'] - df_all['flow2010'])/df_all['flow2010'])
df_all['increase_rate_2010_2015'] = df_all['increase_rate_2010_2015'].fillna(0)

In [33]:
df_all.loc[~np.isfinite(df_all['increase_rate_2010_2015']), 'increase_rate_2010_2015'] = float(0)

### 4.3 2015 - 2019 increase rate 

In [34]:
# Calcualte migration flow increase rate
df_all['increase_rate_2015_2019'] = 100 * ((df_all['flow2019'] - df_all['flow2015'])/df_all['flow2015'])
df_all['increase_rate_2015_2019'] = df_all['increase_rate_2015_2019'].fillna(0)

In [35]:
df_all.loc[~np.isfinite(df_all['increase_rate_2015_2019']), 'increase_rate_2015_2019'] = float(0)

In [36]:
df_all.head(2)

Destination   Origin  flow2000  flow2005  flow2010  flow2015  flow2019  \
0  Afghanistan  Albania       0.0       0.0       0.0       0.0       0.0   
1  Afghanistan  Algeria       0.0       0.0       0.0       0.0       0.0   

   increase_rate_2000_2005  increase_rate_2005_2010  increase_rate_2010_2015  \
0                      0.0                      0.0                      0.0   
1                      0.0                      0.0                      0.0   

   increase_rate_2015_2019  
0                      0.0  
1                      0.0

In [37]:
df_all.iloc[168:170]

Destination               Origin  flow2000  flow2005  flow2010  flow2015  \
168  Afghanistan  Republic of Moldova       0.0       0.0       0.0       0.0   
169  Afghanistan              Réunion       0.0       0.0       0.0       0.0   

     flow2019  increase_rate_2000_2005  increase_rate_2005_2010  \
168       0.0                      0.0                      0.0   
169       0.0                      0.0                      0.0   

     increase_rate_2010_2015  increase_rate_2015_2019  
168                      0.0                      0.0  
169                      0.0                      0.0

## 5 Export edgelist csv file 

In [38]:
df_all.to_csv('EdgeList_All.csv', index=False)

# Part II: Nodes csv 

## 1 Create the region feature (22 regions)

In [39]:
def get_txt_paths(path):
    txt_paths = []
    txt_file_names = []
    
    for file in os.listdir(path):
        # txt paths
        single_file_path = os.path.join(path, file)
        txt_paths.append(single_file_path)
        
        # txt file names
        file_name = file[:-4]
        txt_file_names.append(file_name)
    
    return txt_paths, txt_file_names

path_name = "./RegionData"
txt_path, txt_file_name = get_txt_paths(path_name)
txt_path, txt_file_name

(['./RegionData\\Australia New Zealand.txt',
  './RegionData\\Caribbean.txt',
  './RegionData\\Central America.txt',
  './RegionData\\Central Asia.txt',
  './RegionData\\Eastern Africa.txt',
  './RegionData\\Eastern Asia.txt',
  './RegionData\\Eastern Europe.txt',
  './RegionData\\Melanesia.txt',
  './RegionData\\Micronesia.txt',
  './RegionData\\Middle Africa.txt',
  './RegionData\\Northern Africa.txt',
  './RegionData\\Northern America.txt',
  './RegionData\\Northern Europe.txt',
  './RegionData\\Polynesia.txt',
  './RegionData\\South America.txt',
  './RegionData\\South Eastern Asia.txt',
  './RegionData\\Southern Africa.txt',
  './RegionData\\Southern Asia.txt',
  './RegionData\\Southern Europe.txt',
  './RegionData\\Western Africa.txt',
  './RegionData\\Western Asia.txt',
  './RegionData\\Western Europe.txt'],
 ['Australia New Zealand',
  'Caribbean',
  'Central America',
  'Central Asia',
  'Eastern Africa',
  'Eastern Asia',
  'Eastern Europe',
  'Melanesia',
  'Micronesia',
  '

In [40]:
# Create a dictionary to store region names and corresponding country names
# Create a dict with region names as the keys
region_country_dict = {}
for region in txt_file_name:
    region_country_dict[region] = []
region_country_dict  

{'Australia New Zealand': [],
 'Caribbean': [],
 'Central America': [],
 'Central Asia': [],
 'Eastern Africa': [],
 'Eastern Asia': [],
 'Eastern Europe': [],
 'Melanesia': [],
 'Micronesia': [],
 'Middle Africa': [],
 'Northern Africa': [],
 'Northern America': [],
 'Northern Europe': [],
 'Polynesia': [],
 'South America': [],
 'South Eastern Asia': [],
 'Southern Africa': [],
 'Southern Asia': [],
 'Southern Europe': [],
 'Western Africa': [],
 'Western Asia': [],
 'Western Europe': []}

In [41]:
# Read the countries in each txt file
# and Append the list of countries to the corresponding key
path_name = "./RegionData"
for region in txt_file_name:
    new_path = path_name + "\\" + region + ".txt"
    txt_file = open(new_path, encoding='utf-8')
    txt = txt_file.read().split("\n")
    txt_file.close()
    region_country_dict[region] = txt

region_country_dict

{'Australia New Zealand': ['Australia', 'New Zealand'],
 'Caribbean': ['Anguilla',
  'Antigua and Barbuda',
  'Aruba',
  'Bahamas',
  'Barbados',
  'British Virgin Islands',
  'Bonaire, Sint Eustatius and Saba',
  'Cayman Islands',
  'Cuba',
  'Curaçao',
  'Dominica',
  'Dominican Republic',
  'Grenada',
  'Guadeloupe',
  'Haiti',
  'Jamaica',
  'Martinique',
  'Montserrat',
  'Puerto Rico',
  'Saint Kitts and Nevis',
  'Saint Lucia',
  'Saint Vincent and the Grenadines',
  'Sint Maarten (Dutch part)',
  'Trinidad and Tobago',
  'Turks and Caicos Islands',
  'United States Virgin Islands'],
 'Central America': ['Belize',
  'Costa Rica',
  'El Salvador',
  'Guatemala',
  'Honduras',
  'Mexico',
  'Nicaragua',
  'Panama'],
 'Central Asia': ['Kazakhstan',
  'Kyrgyzstan',
  'Tajikistan',
  'Turkmenistan',
  'Uzbekistan'],
 'Eastern Africa': ['Burundi',
  'Comoros',
  'Djibouti',
  'Eritrea',
  'Ethiopia',
  'Kenya',
  'Madagascar',
  'Malawi',
  'Mauritius',
  'Mayotte',
  'Mozambique',
  

In [42]:
df_nodes = df_2019[["Destination"]].sort_values(by=['Destination'], ascending=True)
df_nodes = df_nodes.reset_index(drop=True)
df_nodes = df_nodes.rename(columns={"Destination":"Country"})

# Transform dict into a dataframe 
df_region = pd.DataFrame.from_dict(region_country_dict, orient='index')
df_region = df_region.transpose()

In [43]:
def generate_region_dfs(region_name, rc_dict, DF):
    # test on one column first
    region_name_1 = region_name

    # get the value of the key and calculate the length of that value list
    length = len(region_country_dict[region_name_1])

    # get the rows of the chosen column
    b = df_region[[region_name_1]][:length]

    # create a new column "Region" with the corresponding region name
    col_name = list(b.columns)
    b['Region'] = col_name[0]
    
    return b

In [44]:
# Using a loop to concat all sub-region dfs
df_list = ["df1", "df2", "df3", "df4", "df5", "df6", "df7", "df8", "df9", "df10", "df11", "df12", "df13", "df14", "df15", "df16", "df17", "df18", "df19", "df20", "df21", "df22"]

txt_file_name_short = txt_file_name[1:]
i = 0
for i in range(0, 22):
    region_name = txt_file_name[i]
    
    df2 = generate_region_dfs(region_name, region_country_dict, df_region)
    df2 = df2.rename(columns={region_name:"Country"})
    df_list[i] = df2
    i += 1

result = pd.concat(df_list).reset_index(drop = True)
result.head()

Country                 Region
0            Australia  Australia New Zealand
1          New Zealand  Australia New Zealand
2             Anguilla              Caribbean
3  Antigua and Barbuda              Caribbean
4                Aruba              Caribbean

## 2 Create a second region feature (7 regions)

In [45]:
# Africa = {"Sub_Saharan Arica": ["Eastern Africa", "Middle Africa", "Southern Africa", "Western Africa"]}
# AfricaAsia = {"Northern Africa and Western Asia": ["Northern Africa", "Western Asia"]}
# CSAsia = {"Central and Southern Asia": ["Central Asia", "Southern Asia"]}
# ESAsia = {"Eastern and South Eastern Asia": ["Eastern Asia", "South Eastern Asia"]}
# America = {"Latin America and The Caribbean": ["Caribbean", "Central America", "South America"]}
# Oceania = {"Oceania": ["Australia New Zealand", "Melanesia", "Micronesia", "Polynesia"]}
# Europe = {"Europe and Northern America": ["Eastern Europe", "Northern Europe",
#          "Southern Europe", "Western Europe", "Northern America"]}

In [46]:
result['UN_Region'] = None
result.head()

Country                 Region UN_Region
0            Australia  Australia New Zealand      None
1          New Zealand  Australia New Zealand      None
2             Anguilla              Caribbean      None
3  Antigua and Barbuda              Caribbean      None
4                Aruba              Caribbean      None

In [47]:
result2 = result.copy()

In [48]:
result2['UN_Region'] = np.where(result2['Region'] == "Eastern Africa", "Sub_Saharan Africa", 
                        np.where(result2['Region'] == "Middle Africa", "Sub_Saharan Africa",
                        np.where(result2['Region'] == "Southern Africa", "Sub_Saharan Africa",
                        np.where(result2['Region'] == "Western Africa", "Sub_Saharan Africa",
                        np.where(result2['Region'] == "Northern Africa", "Northern Africa and Western Asia",
                        np.where(result2['Region'] == "Western Asia", "Northern Africa and Western Asia",
                        np.where(result2['Region'] == "Central Asia", "Central and Southern Asia",
                        np.where(result2['Region'] == "Southern Asia", "Central and Southern Asia",
                        np.where(result2['Region'] == "Eastern Asia", "Eastern and South Eastern Asia",
                        np.where(result2['Region'] == "South Eastern Asia", "Eastern and South Eastern Asia",
                        np.where(result2['Region'] == "Caribbean", "Latin America and The Caribbean",
                        np.where(result2['Region'] == "Central America", "Latin America and The Caribbean",
                        np.where(result2['Region'] == "South America", "Latin America and The Caribbean",
                        np.where(result2['Region'] == "Australia New Zealand", "Oceania",
                        np.where(result2['Region'] == "Melanesia", "Oceania",
                        np.where(result2['Region'] == "Micronesia", "Oceania",
                        np.where(result2['Region'] == "Polynesia", "Oceania",
                        np.where(result2['Region'] == "Eastern Europe", "Europe and Northern America",
                        np.where(result2['Region'] == "Northern Europe", "Europe and Northern America",
                        np.where(result2['Region'] == "Southern Europe", "Europe and Northern America",
                        np.where(result2['Region'] == "Western Europe", "Europe and Northern America",
                        np.where(result2['Region'] == "Northern America", "Europe and Northern America",
                                 "no match"))))))))))))))))))))))


In [49]:
result2.head()

Country                 Region                        UN_Region
0            Australia  Australia New Zealand                          Oceania
1          New Zealand  Australia New Zealand                          Oceania
2             Anguilla              Caribbean  Latin America and The Caribbean
3  Antigua and Barbuda              Caribbean  Latin America and The Caribbean
4                Aruba              Caribbean  Latin America and The Caribbean

## 3 Process influenza data

### 3.1 Combine all influenza csv files 

In [50]:
# change current directory path
os.chdir("./Influenza_165")

In [51]:
# set extension
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

# combine all files in the list
combined_csv = pd.concat([pd.read_csv(f, header = 2, engine="python") for f in all_filenames], sort=False)

# export to csv
#combined_csv.to_csv("a_combined_data.csv", index=False, encoding='utf-8-sig')

# reference: https://www.freecodecamp.org/news/how-to-combine-multiple-csv-files-with-8-lines-of-code-265183e0854/

In [52]:
df = combined_csv.copy()

In [53]:
df.columns

Index(['Country', 'WHOREGION', 'FLUREGION', 'Year', 'Week', 'SDATE', 'EDATE',
       'SPEC_RECEIVED_NB', 'SPEC_PROCESSED_NB', 'AH1', 'AH1N12009', 'AH3',
       'AH5', 'ANOTSUBTYPED', 'INF_A', 'BYAMAGATA', 'BVICTORIA',
       'BNOTDETERMINED', 'INF_B', 'ALL_INF', 'ALL_INF2', 'TITLE'],
      dtype='object')

### 3.2 Calculate influenza cases by country and by year 

In [54]:
# subset only useful columns
df = df[['Country', 'Year', 'ALL_INF']]

# fill nan with 0
df_orig = df.copy()
df = df.fillna(0)

# apply group by function to the dataframe
df_allinf = df.groupby(['Country', 'Year'])['ALL_INF'].sum()
df_allinf = pd.DataFrame(df_allinf).reset_index()
#df_allinf.head(2)

# check null
df_allinf.isnull().any()

Country    False
Year       False
ALL_INF    False
dtype: bool

In [55]:
# Pivot table
df_allinf = df_allinf.pivot(index = 'Country', columns = 'Year',
                            values = 'ALL_INF').add_prefix('Influenza').reset_index()
df_allinf.head(2)

Year      Country  Influenza2005  Influenza2006  Influenza2007  Influenza2008  \
0     Afghanistan            0.0            0.0            0.0            0.0   
1         Albania            0.0            0.0            0.0            0.0   

Year  Influenza2009  Influenza2010  Influenza2011  Influenza2012  \
0             283.0           26.0            5.0            2.0   
1               0.0           10.0          133.0          122.0   

Year  Influenza2013  Influenza2014  Influenza2015  Influenza2016  \
0               0.0            0.0           13.0          257.0   
1              97.0          157.0           67.0          314.0   

Year  Influenza2017  Influenza2018  Influenza2019  
0             108.0          187.0          278.0  
1             136.0          350.0          527.0

In [56]:
df_allinf.isnull().any()

Year
Country          False
Influenza2005     True
Influenza2006     True
Influenza2007     True
Influenza2008     True
Influenza2009     True
Influenza2010    False
Influenza2011    False
Influenza2012    False
Influenza2013    False
Influenza2014    False
Influenza2015    False
Influenza2016    False
Influenza2017    False
Influenza2018    False
Influenza2019    False
dtype: bool

In [57]:
df_allinf = df_allinf.fillna(0)
df_allinf.isnull().any()

Year
Country          False
Influenza2005    False
Influenza2006    False
Influenza2007    False
Influenza2008    False
Influenza2009    False
Influenza2010    False
Influenza2011    False
Influenza2012    False
Influenza2013    False
Influenza2014    False
Influenza2015    False
Influenza2016    False
Influenza2017    False
Influenza2018    False
Influenza2019    False
dtype: bool

In [58]:
df_allinf.dtypes

Year
Country           object
Influenza2005    float64
Influenza2006    float64
Influenza2007    float64
Influenza2008    float64
Influenza2009    float64
Influenza2010    float64
Influenza2011    float64
Influenza2012    float64
Influenza2013    float64
Influenza2014    float64
Influenza2015    float64
Influenza2016    float64
Influenza2017    float64
Influenza2018    float64
Influenza2019    float64
dtype: object

### 3.3 Unify country names

In [59]:
# find the country name that used by UN
UN_cote_divore = region_country_dict['Western Africa'][3]
UN_cote_divore

"Côte d'Ivoire"

In [60]:
# locate the position of the special letter country name in the influenza df
df_allinf.iloc[40:42]

Year         Country  Influenza2005  Influenza2006  Influenza2007  \
40           Czechia           46.0          161.0          262.0   
41    CÃ´te d'Ivoire            0.0            0.0            0.0   

Year  Influenza2008  Influenza2009  Influenza2010  Influenza2011  \
40            187.0          990.0           72.0          427.0   
41              0.0          344.0          214.0          417.0   

Year  Influenza2012  Influenza2013  Influenza2014  Influenza2015  \
40            249.0         1030.0           56.0          725.0   
41            261.0          351.0          212.0          287.0   

Year  Influenza2016  Influenza2017  Influenza2018  Influenza2019  
40            486.0          368.0          575.0          276.0  
41            360.0          337.0          262.0          458.0

In [61]:
# change the corresponding name used by WHO to that adopted by UN
df_allinf.loc[41, ['Country']] = UN_cote_divore

# check whether it has been changed

In [62]:
# check whether it has been changed
df_allinf.iloc[40:42]

Year        Country  Influenza2005  Influenza2006  Influenza2007  \
40          Czechia           46.0          161.0          262.0   
41    Côte d'Ivoire            0.0            0.0            0.0   

Year  Influenza2008  Influenza2009  Influenza2010  Influenza2011  \
40            187.0          990.0           72.0          427.0   
41              0.0          344.0          214.0          417.0   

Year  Influenza2012  Influenza2013  Influenza2014  Influenza2015  \
40            249.0         1030.0           56.0          725.0   
41            261.0          351.0          212.0          287.0   

Year  Influenza2016  Influenza2017  Influenza2018  Influenza2019  
40            486.0          368.0          575.0          276.0  
41            360.0          337.0          262.0          458.0

In [63]:
# replace names in df_allinf
# certain country names adopted by WHO is different from that of UN

df_allinf_unified = df_allinf.replace({"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                  "Lao People's Democratic Republic": "Lao People's Democratic Republic",
                  "Turks and Caicos is.": "Turks and Caicos Islands",
                  "Democratic People's Republic of Korea": "Dem. People's Republic of Korea"})

In [64]:
# check
df_allinf_unified.iloc[153]

# The following results show that Turks and Caicos Is.'s name has not been changed yet

Year
Country          Turks and Caicos Is.
Influenza2005                       0
Influenza2006                       0
Influenza2007                       0
Influenza2008                       0
Influenza2009                       0
Influenza2010                       0
Influenza2011                       0
Influenza2012                       0
Influenza2013                       0
Influenza2014                       0
Influenza2015                       0
Influenza2016                       0
Influenza2017                       0
Influenza2018                       0
Influenza2019                      12
Name: 153, dtype: object

In [65]:
# change name by hand
df_allinf_unified.at[153, 'Country'] = "Turks and Caicos Islands"
df_allinf_unified.iloc[153]

Year
Country          Turks and Caicos Islands
Influenza2005                           0
Influenza2006                           0
Influenza2007                           0
Influenza2008                           0
Influenza2009                           0
Influenza2010                           0
Influenza2011                           0
Influenza2012                           0
Influenza2013                           0
Influenza2014                           0
Influenza2015                           0
Influenza2016                           0
Influenza2017                           0
Influenza2018                           0
Influenza2019                          12
Name: 153, dtype: object

## 4 Merge the influenza df and the country nodes df 

In [66]:
# merge
df_merge = pd.merge(result2, df_allinf_unified, how = 'left', 
                    left_on = 'Country', right_on = 'Country').fillna(0)
df_merge.head(2)

Country                 Region UN_Region  Influenza2005  Influenza2006  \
0    Australia  Australia New Zealand   Oceania          712.0          259.0   
1  New Zealand  Australia New Zealand   Oceania          269.0          285.0   

   Influenza2007  Influenza2008  Influenza2009  Influenza2010  Influenza2011  \
0         1355.0          768.0         5107.0         1252.0         2040.0   
1          207.0          548.0         1542.0          326.0         1218.0   

   Influenza2012  Influenza2013  Influenza2014  Influenza2015  Influenza2016  \
0         4798.0         2002.0         3473.0         3622.0         6705.0   
1         2261.0         2185.0         3418.0         5102.0          294.0   

   Influenza2017  Influenza2018  Influenza2019  
0        10509.0         4264.0        14002.0  
1          945.0          475.0          957.0

In [67]:
len(df_merge)

232

In [68]:
df_merge.isnull().sum().sum()

0

## 5 Export country node csv file

In [69]:
# set directory path
path = "C:/Users/Winnie/Documents/NeilJohnson/Migration/Version_7_March_31/"

# export csv files for 232 countres
df_merge.to_csv(path + 'CountryNodes_All.csv', index=False)

In [70]:
len(df_merge['Country'].unique())

232